In [3]:
import librosa
import numpy as np
from keras.models import load_model


In [53]:
# Modeli yükleyin
model_path = 'C:\\Users\\osman\\Desktop\\Notebook Workspace\\emotion_model.h5'  # Burada model yolunu belirtin
model = load_model(model_path)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 20, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 20, 64)              │          12,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 20, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 10, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 10, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 10, 128)             │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 10, 128)             │          49,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 5, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 5, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 6)                   │           7,686 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 242,120 (945.79 KB)

 Trainable params: 242,118 (945.77 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [93]:
import sounddevice as sd
from scipy.io.wavfile import write
import numpy as np

# Function to record audio and save it as a WAV file
def record_audio(duration=2.5, sample_rate=22050, file_name='recorded_audio.wav'):
    print("Recording audio...")
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='float32')
    sd.wait()  # Wait until recording is finished
    write(file_name, sample_rate, (audio_data * 32767).astype(np.int16))  # Save as WAV
    print(f"Recording finished. Saved as {file_name}")
    return file_name

# Example usage:
file_name = record_audio(duration=2.5, sample_rate=22050, file_name='recorded_audio.wav')


Recording audio...
Recording finished. Saved as recorded_audio.wav


In [94]:
import numpy as np
import pickle
import librosa
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
# Google Drive'daki scaler.pkl dosyasını yükle
# Load the scaler from the saved .pkl file
from joblib import load
scaler_file = 'C:\\Users\\osman\\Desktop\\Notebook Workspace\\scaler.pkl'  # Update this path if needed
try:
    with open(scaler_file, 'rb') as f:
        loaded_scaler = load('scaler.pkl')
        print("Scaler loaded successfully.")
except FileNotFoundError:
    print(f"Error: Scaler file not found at {scaler_file}. Make sure you saved it.")

# Extract features from audio file
def extract_features_for_test(file_path, loaded_scaler, sample_rate=22050):
    # Load audio file
    data, sr = librosa.load(file_path, duration=2.5, offset=0.6)

    # Extract MFCC features (20 coefficients)
    mfcc = librosa.feature.mfcc(y=data, sr=sr, n_mfcc=20)

    # Compute the mean of MFCC features
    mfcc_processed = np.mean(mfcc.T, axis=0)

    # Reshape for scaling
    mfcc_processed = mfcc_processed.reshape(1, -1)

    # Use the loaded scaler to scale the features
    mfcc_scaled = loaded_scaler.transform(mfcc_processed)

    # Reshape to match model input (same as during training)
    mfcc_scaled = np.expand_dims(mfcc_scaled, axis=2)

    return mfcc_scaled

# Test the model on the audio file
def test_audio_model(file_path, model, loaded_scaler):
    # Extract and preprocess features
    features = extract_features_for_test(file_path, loaded_scaler)

    # Make prediction
    prediction = model.predict(features)

    # Get the predicted class
    predicted_class = np.argmax(prediction, axis=1)

    # Decode the class into emotion labels
    emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']
    predicted_emotion = emotion_labels[predicted_class[0]]

    return predicted_emotion, prediction[0]

# Diagnosing predictions for multiple audio files
def diagnose_model_prediction(file_paths, model, loaded_scaler):
    results = []
    for path in file_paths:
        predicted_emotion, confidence_scores = test_audio_model(path, model, loaded_scaler)
        results.append({
            'path': path,
            'predicted_emotion': predicted_emotion,
            'confidence_scores': confidence_scores
        })
    return results

# Example usage
file_paths = [
    'C:\\Users\\osman\\Desktop\\Notebook Workspace\\recorded_audio.wav',  # Update with your actual file path
]

# Run diagnostic predictions
diagnostic_results = diagnose_model_prediction(file_paths, model, loaded_scaler)

# Print results
for result in diagnostic_results:
    print(f"Path: {result['path']}")
    print(f"Predicted Emotion: {result['predicted_emotion']}")
    print("Confidence Scores:")
    emotion_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad']
    for label, score in zip(emotion_labels, result['confidence_scores']):
        print(f"{label}: {score:.4f}")
    print("\n")

Scaler loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Path: C:\Users\osman\Desktop\Notebook Workspace\recorded_audio.wav
Predicted Emotion: sad
Confidence Scores:
angry: 0.0000
disgust: 0.0003
fear: 0.3387
happy: 0.0001
neutral: 0.0002
sad: 0.6608


